In [25]:
import pandas as pd

import nltk
# nltk.download('stopwords')
# nltk.download('punkt_tab')

from nltk.corpus import stopwords
from gensim.corpora import Dictionary
from gensim.models import TfidfModel

from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

import json


# Data Modeling 
# pick two topics to compare
# develop a way to identify which topic a post is talking about 
# check occurences of different topics of conversation over time
#   could these different conversation topics appear during a game period more often?


In [29]:

def text2tokens(text):
	stop_words = set(stopwords.words('english'))
	ps = PorterStemmer()
	text = text.lower()
	textList = word_tokenize(text)
	textList = [word for word in textList if word not in stop_words and len(word) >= 3]

	textList = [ps.stem(word) for word in textList]
	return textList
    

def gen_bow(df, column):
	df['tokens'] = df[column].apply(text2tokens)
	dct = Dictionary(df['tokens'])
	dct.filter_extremes(no_below=5, no_above=0.5)
	df['bow'] = df['tokens'].apply(dct.doc2bow)
	worddict = {}

	for i in range(len(df)):
		for token in df.at[i,'bow']:
			if token in list(worddict.keys()):
				worddict[token] += 1
			else: 
				worddict[token] = 1

	df.drop('tokens', axis=1, inplace=True)	
	return df, worddict

def get_post(jobj):
	data_obj = jobj[0]['data']['children'][0]['data']
	txt = data_obj['selftext']
	author = data_obj['author']
	time = data_obj['created_utc']
	
	return {
		'author': author,
		'created_utc': time,
		'text': txt
	}

def get_comment(jobj):
	comments = []
	for comment in jobj[1]['data']['children']:
		if 'author' in comment['data']\
		and 'created_utc' in comment['data']\
		and 'body' in comment['data']:
			cobj = {
				'author': comment['data']['author'],
				'created_utc': comment['data']['created_utc'],
				'text': comment['data']['body']
			}
			comments.append(cobj)

	return comments

# print(text2tokens("In this offering, one only has to view the current Westminster Kennel Club's annual offering to see the parallels."))



In [30]:
orioles_posts = []

with open('orioles.json', 'r') as f:
	lines = f.readlines()

for line in lines:
	data_obj = json.loads(line)
	dobj = get_post(data_obj)
	orioles_posts.append(dobj)
	comments = get_comment(data_obj)
	orioles_posts.extend(comments)

orioles_df = pd.DataFrame(orioles_posts)
	# data = json.load(f)
	# orioles_df = pd.json_normalize(data, record_path='items')
	# orioles_df


JSONDecodeError: Expecting value: line 2 column 1 (char 408858)